# Generation of figure for inclusion in the introduction page in the documentation

<span style='color:red'>*This is not a tutorial notebook.*</span> 

It is a tool for generating the figures included in `docs/source/introduction.rst`.

If you you are looking for an introductory tutorial, see `uranography.ipynb`.

In [ ]:
import warnings
import numpy as np
import healpy as hp
import bokeh
import colorcet as cc
import pandas as pd
import panel as pn
from astropy.time import Time
import sqlite3
import urllib.request
import astropy.coordinates
import bs4

import healsparse as hsp

from uranography.api import (
    Planisphere,
    ArmillarySphere,
    MollweideMap,
    HorizonMap,
    split_healpix_by_resolution,
    load_bright_stars,
    CameraFootprintPerimeter,
)

In [ ]:
pn.extension()

In [ ]:
night = Time("2026-05-30", location=astropy.coordinates.EarthLocation.of_site("Cerro Pachon"))

In [ ]:
warnings.filterwarnings('ignore', message=r'.*Tried to get polar motions for times after IERS data is valid.*')

# Load stars

In [ ]:
sun_coords = astropy.coordinates.get_sun(night)
moon_coords = astropy.coordinates.get_moon(night)
sun_coords, moon_coords

In [ ]:
stars_fname = "bsc5.dat.gz"
if not os.path.isfile(stars_fname):
    urllib.request.urlretrieve("http://tdc-www.harvard.edu/catalogs/bsc5.dat.gz", stars_fname)

Now, load the stars:

In [ ]:
stars = load_bright_stars(stars_fname)
len(stars)

This is way too many stars. Only consider the brightest:

In [ ]:
stars.query("Vmag<3.5", inplace=True)
len(stars)

# Multiple connected views

In [ ]:
data_source = {}

asphere_plot = bokeh.plotting.figure(
    frame_width=768,
    frame_height=384,
    match_aspect=True,
    title="Armillary Sphere",
)
asphere = ArmillarySphere(mjd=night.mjd, plot=asphere_plot)
asphere.add_mjd_slider()

data_source['sun'] = asphere.add_marker(
    sun_coords.ra.deg,
    sun_coords.dec.deg,
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "orange", "legend_label": "Sun"},
)
data_source['moon'] = asphere.add_marker(
    moon_coords.ra.deg,
    moon_coords.dec.deg,
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "lightgray", "legend_label": "Moon"},
)

# Scale the size of the star markers with the magnitude of the stars
stars["glyph_size"] = 5 * (1.01 - stars["Vmag"] / stars["Vmag"].max())

# Actually add the stars
data_source['stars'] = asphere.add_stars(stars, mag_limit_slider=False, star_kwargs={"color": "black"})


asphere.add_graticules()
asphere.add_ecliptic(legend_label="Ecliptic")
asphere.add_galactic_plane(legend_label="Galactic plane")
data_source["horizon"] = asphere.add_horizon(line_kwargs={"legend_label": "Horizon"})
data_source["high_X"] = asphere.add_horizon(
    zd=70, line_kwargs={"color": "red", "line_width": 2, "legend_label": "ZD=70" + u'\N{DEGREE SIGN}'}
)
asphere.plot.add_layout(asphere.plot.legend[0], "left")
asphere.show()
asphere_fname = bokeh.plotting.save(asphere.figure, filename='asphere.html')

with open(asphere_fname, 'r', encoding="utf-8") as in_file:
    html_text = in_file.read()

updated_html = html_text.replace(
    "<script",
    """<script src="https://unpkg.com/gpu.js@latest/dist/gpu-browser.min.js"></script>
    <script""",
    1)
    
with open(asphere_fname, 'w', encoding='utf-8') as out_file:
    out_file.write(updated_html)
    
if False:
    # Edit the exported file to source the needed gpujs library
    with open(asphere_fname) as in_file:
        asphere_soup = bs4.BeautifulSoup(in_file.read())

    gpujs_source = asphere_soup.new_tag("source", src="https://unpkg.com/gpu.js@latest/dist/gpu-browser.min.js", type="text/javascript")
    asphere_soup.head.append(gpujs_source)

    with open(asphere_fname, 'w', encoding='utf-8') as out_file:
        out_file.write(str(asphere_soup))

In [ ]:
msphere_plot = bokeh.plotting.figure(
    frame_width=768,
    frame_height=384,
    match_aspect=False,
    title="Mollweide",
)
msphere = MollweideMap(mjd=night.mjd, plot=msphere_plot)
msphere.add_mjd_slider()
msphere.sliders['mjd'] = asphere.sliders['mjd']

msphere.add_marker(
    data_source=data_source['sun'],
    name="Sun",
    glyph_size=15,
    circle_kwargs={"color": "orange", "legend_label": "Sun"},
)

msphere.add_marker(
    data_source=data_source['moon'],
    name="Moon",
    glyph_size=15,
    circle_kwargs={"color": "lightgray", "legend_label": "Moon"},
)

msphere.plot.star(
            x=msphere.proj_transform("x", data_source['stars']),
            y=msphere.proj_transform("y", data_source['stars']),
            size="glyph_size",
            source=data_source['stars'],
            color='black',
        )

msphere.add_graticules(
    graticule_kwargs={
        "min_decl": -80,
        "max_decl": 80,
        "decl_space": 20,
        "min_ra": 0,
        "max_ra": 360,
        "ra_space": 30,
    },
    line_kwargs={"color": "lightgray"},
)
# HACK to make the RA=180 graticule appear both on the left and right
msphere.add_graticules(
    graticule_kwargs={
        "min_decl": -80,
        "max_decl": 80,
        "decl_space": 160,
        "min_ra": 180 - 4,
        "max_ra": 180 + 5,
        "ra_space": 8,
    },
    line_kwargs={"color": "lightgray"},
)

msphere.add_ecliptic(legend_label="Ecliptic")
msphere.add_galactic_plane(legend_label="Galactic plane")
msphere.add_horizon(data_source=data_source["horizon"], line_kwargs={"legend_label": "Horizon"})
msphere.add_horizon(
    data_source=data_source["high_X"], line_kwargs={"color": "red", "line_width": 2, "legend_label": "ZD=70" + u'\N{DEGREE SIGN}'}
)

msphere.plot.add_layout(msphere.plot.legend[0], "left")
msphere.show()
bokeh.plotting.save(msphere.figure, filename='mollweide.html')